In [162]:
# import everything ml
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import re
import string
from wordcloud import WordCloud
import nltk
from collections import Counter
import contractions
from tqdm import tqdm
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [163]:
true_dataset = pd.read_csv("./data/fakenews/True.csv")
false_dataset = pd.read_csv("./data/fakenews/Fake.csv")

In [164]:
def combine_and_randomize_datasets(true_dataset, false_dataset):
    true_arr = np.ones(len(true_dataset))
    false_arr = np.zeros(len(false_dataset))

    true_dataset['real'] = true_arr
    false_dataset['real'] = false_arr
    combined_dataset = pd.concat([true_dataset, false_dataset], ignore_index=True)
    combined_dataset = combined_dataset.sample(frac=1).reset_index(drop=True)

    return combined_dataset

combined = combine_and_randomize_datasets(true_dataset, false_dataset)

In [165]:


def clean_text(text):
    text = contractions.fix(text)                         # Expand contractions
    text = text.encode("ascii", errors="ignore").decode() # Remove non-ASCII chars
    text = text.lower()                                   # Lowercase
    text = re.sub(r"http\S+|www.\S+", "", text)           # Remove URLs
    text = re.sub(r"<.*?>", "", text)                     # Remove HTML tags
    text = re.sub(r"[^a-z\s]", "", text)                  # Keep only letters and spaces
    text = re.sub(r"\b\w{1}\b", "", text)                 # Remove single-letter words
    text = re.sub(r"\s+", " ", text).strip()              # Remove extra spaces
    words = text.split()
    # words = [word for word in words if word not in stop_words]  # Remove stopwords
    return " ".join(words)



In [166]:
words_dict = {}

# def process_words_into_dict(text, words_dict):
#     for word in text.split():
#         if word not in words_dict:
#             words_dict[word] = 1
#         else:
#             words_dict[word] += 1

# for text in combined['text']:
#     text = clean_text(text)
# tfidf = calculate_tfidf(combined['text'])

X = combined['text']
y = combined['real'].to_list()

test_x = X[:20]
X = X[20:]

test_y=y[:20]
y=y[20:]

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words='english')
X_tfidf = vectorizer.fit_transform(X)  # shape (n_samples, n_features)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# --- Convert to dense and then to PyTorch tensors
X_train_tensor = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [167]:
# --- PyTorch Dataset
class NewsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y.unsqueeze(1)  # Per sample

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = NewsDataset(X_train_tensor, y_train_tensor)
test_dataset = NewsDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [168]:
# --- Simple Feedforward Model
# class FakeNewsClassifier(nn.Module):
#     def __init__(self, input_dim):
#         super().__init__()
#         self.fc1 = nn.Linear(input_dim, 100)
#         self.fc2 = nn.Linear(100, 1)

#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = torch.sigmoid(self.fc2(x))
#         return x

class FakeNewsClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 1000),
            nn.ReLU(),
            nn.Linear(1000,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.classifier(x)
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
model = FakeNewsClassifier(input_dim=5000).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [169]:
# --- Training Loop
for epoch in range(50):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model
        outputs = model(batch_X)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")


Epoch 1 - Loss: 71.5659
Epoch 2 - Loss: 14.2064
Epoch 3 - Loss: 6.7422
Epoch 4 - Loss: 5.8403
Epoch 5 - Loss: 4.2146
Epoch 6 - Loss: 2.8628
Epoch 7 - Loss: 1.0371
Epoch 8 - Loss: 1.4660
Epoch 9 - Loss: 3.5881
Epoch 10 - Loss: 1.9631
Epoch 11 - Loss: 0.8713
Epoch 12 - Loss: 0.5663
Epoch 13 - Loss: 0.6283
Epoch 14 - Loss: 0.4269
Epoch 15 - Loss: 0.5722
Epoch 16 - Loss: 1.0008
Epoch 17 - Loss: 1.7675
Epoch 18 - Loss: 3.3333
Epoch 19 - Loss: 1.4838
Epoch 20 - Loss: 0.6453
Epoch 21 - Loss: 0.5984
Epoch 22 - Loss: 0.3890
Epoch 23 - Loss: 0.4235
Epoch 24 - Loss: 0.4727
Epoch 25 - Loss: 0.3564
Epoch 26 - Loss: 0.4790
Epoch 27 - Loss: 0.3820
Epoch 28 - Loss: 0.5263
Epoch 29 - Loss: 0.4679
Epoch 30 - Loss: 0.4026
Epoch 31 - Loss: 0.3833
Epoch 32 - Loss: 0.4551
Epoch 33 - Loss: 0.7018
Epoch 34 - Loss: 1.4880
Epoch 35 - Loss: 2.1392
Epoch 36 - Loss: 3.5972
Epoch 37 - Loss: 0.9349
Epoch 38 - Loss: 0.4959
Epoch 39 - Loss: 0.5643
Epoch 40 - Loss: 0.5420
Epoch 41 - Loss: 0.4308
Epoch 42 - Loss: 0.4088

In [183]:
# --- Evaluation
model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)  # Ensure inputs are on model's device
    y_test_tensor = y_test_tensor.to(device)  # Ensure labels are on the same device

    preds = model(X_test_tensor).squeeze()
    #print(f"average pred: {preds.mean().item()}")
    preds_label = (preds > 0.5).int()
    accuracy = (preds_label == y_test_tensor.int()).float().mean()
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9904


In [182]:
def predict_news_article(article_text, vectorizer, model):
    model.eval()
    with torch.no_grad():
        # Vectorize input string
        tfidf_vector = vectorizer.transform([article_text])
        tfidf_tensor = torch.tensor(tfidf_vector.toarray(), dtype=torch.float32).to(device)

        # Forward pass
        output = model(tfidf_tensor)
    
        return output, output > 0.5

# test_article = "The government has announced a new plan to build solar-powered roads by 2050..."
# test_article=false_dataset['text'][123]
idx = 9
test_article = test_x[idx]
test_article_pred = test_y[idx]
print(f"pred {test_article_pred} testing article: {test_article}")

prediction, is_true = predict_news_article(test_article, vectorizer, model)
print(f"Prediction: {prediction} \n Is True {is_true} was actually: {test_article_pred>0.5}")

pred 0.0 testing article: If you were to believe Barack Hussein Obama, Hillary Clinton, Bernie Sanders, the Democrats and RINO s in Congress, the only videos you should find on these poor persecuted migrants phones would be of tearful good-byes to families in their beloved home land. Not so fast You only have to watch this one video  to know, these aren t your usual  persecuted  migrants invading Europe.Just watch this video, and you ll see what we mean:https://youtu.be/silF8UT4mTE
Prediction: tensor([[0.]], device='cuda:0') 
 Is True tensor([[False]], device='cuda:0') was actually: False
